# 🌍 Tourist Assistant (Free Version)

This notebook runs the complete Tourist Assistant application using the Google Gemini free tier, Google's free TTS/STT, and the Google Places API.

**Instructions:**
1.  Create a `.env` file in this folder with your Google API keys.
2.  Run the cells in order.
3.  The final cell will launch the web interface.

In [7]:
# This cell installs all the new and existing libraries required for the project.
!pip install gradio python-dotenv pydub pypdf langchain langchain-community faiss-cpu requests
!pip install google-generativeai langchain-google-genai gTTS SpeechRecognition
print("✅ All libraries installed successfully.")

✅ All libraries installed successfully.


In [2]:
# This cell loads all the keys from your .env file.
# Your .env file should contain:
# GOOGLE_API_KEY="YOUR_GEMINI_API_KEY"
# GOOGLE_PLACES_API_KEY="YOUR_GOOGLE_PLACES_KEY"

import os
from dotenv import load_dotenv

load_dotenv(override=True)

# --- Load all Environment Variables ---
print("--- Loading variables from .env file... ---")
google_api_key = os.getenv("GOOGLE_API_KEY")
google_places_api_key = os.getenv('GOOGLE_PLACES_API_KEY')

# --- Verification Step ---
def verify_var(var_name, var_value, is_secret=False):
    if var_value:
        display_value = f"{var_value[:5]}..." if is_secret else var_value
        print(f"✅ {var_name}: {display_value} (Loaded)")
    else:
        print(f"❌ {var_name}: NOT FOUND. Check your .env file.")

print("\n--- Verifying loaded variables ---")
verify_var("GOOGLE_API_KEY", google_api_key, is_secret=True)
verify_var("GOOGLE_PLACES_API_KEY", google_places_api_key, is_secret=True)
print("--------------------------------")

--- Loading variables from .env file... ---

--- Verifying loaded variables ---
✅ GOOGLE_API_KEY: AIzaS... (Loaded)
✅ GOOGLE_PLACES_API_KEY: AIzaS... (Loaded)
--------------------------------


In [3]:
# This cell initializes and tests the core AI models from Google.
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import google.generativeai as genai

print("--- Initializing Google Gemini clients... ---")
try:
    genai.configure(api_key=google_api_key)
    llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.7)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    print("✅ Google Gemini clients initialized successfully.")

    print("\n--- Testing Chat Model (Gemini 2.5 Flash)... ---")
    response = llm.invoke("Hello, world!")
    print(f"✅ SUCCESS! Chat model replied: '{response.content}'")

    print("\n--- Testing Embeddings Model (Gemini 2.5 Flash)... ---")
    vector = embeddings.embed_query("This is a test sentence.")
    print(f"✅ SUCCESS! Embeddings model created a vector of length: {len(vector)}.")
    
except Exception as e:
    print(f"❌ FAILED to initialize or test clients. Error: {e}")
    print("   Please double-check your GOOGLE_API_KEY in the .env file.")

--- Initializing Google Gemini clients... ---
✅ Google Gemini clients initialized successfully.

--- Testing Chat Model (Gemini 2.5 Flash)... ---
✅ SUCCESS! Chat model replied: 'Hello there!

"Hello, world!" back to you! It's a classic for a reason. How can I help you today?'

--- Testing Embeddings Model (Gemini 2.5 Flash)... ---
✅ SUCCESS! Embeddings model created a vector of length: 768.


In [4]:
# This cell tests the connection to the Google Places API.
import requests

print("--- Testing Google Places API... ---")

if not google_places_api_key:
    print("❌ FAILED: GOOGLE_PLACES_API_KEY was not found in your .env file.")
else:
    print(f"✅ Google API Key loaded successfully, starting with: {google_places_api_key[:8]}...")
    test_location = "Eiffel Tower in Paris"
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={test_location}&key={google_places_api_key}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        data = response.json()
        
        if data['status'] == 'OK':
            print("✅ SUCCESS! Google Places API connection is working.")
            print(f"   Test search found: '{data['results'][0]['name']}'.")
        else:
            print(f"❌ FAILED: The API returned a status of '{data['status']}'.")
            if 'error_message' in data:
                print(f"   API Error Message: {data['error_message']}")

    except requests.exceptions.RequestException as e:
        print(f"❌ FAILED: A network error occurred. Error: {e}")

--- Testing Google Places API... ---
✅ Google API Key loaded successfully, starting with: AIzaSyAr...
✅ SUCCESS! Google Places API connection is working.
   Test search found: 'Eiffel Tower'.


In [5]:
# This cell contains the complete, final code for the application.
# It relies on all the clients and variables created in the cells above.

import gradio as gr
import requests
import glob
import os  # <-- ADDED
import threading  # <-- ADDED
from io import BytesIO  # <-- ADDED
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from pydub import AudioSegment
from playsound import playsound
from gtts import gTTS
import speech_recognition as sr
from langchain_core.messages import HumanMessage, AIMessage

# --- RAG Functions and Initialization ---
def read_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() or ""
    return text

def load_knowledge_base():
    os.makedirs("knowledge-base", exist_ok=True)
    pdf_files = glob.glob("knowledge-base/*.pdf")
    if not pdf_files: return None
    all_content = "".join(read_pdf(pdf_file) + "\n\n" for pdf_file in pdf_files)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
    chunks = text_splitter.split_text(all_content)
    # Use the Google embeddings client initialized in the previous cell
    return FAISS.from_texts(chunks, embeddings)

vector_store = load_knowledge_base()
retrieval_chain = None
if vector_store:
    # Use the Google LLM client initialized in the previous cell
    retrieval_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=False
    )
    print("RAG system initialized successfully.")
else:
    print("RAG system not initialized (no PDFs found).")


# --- Core Application Functions ---
def talker(message):
    """Converts text to speech using gTTS and plays it with playsound."""
    
    if not message or message.strip() == "":
        print("No text to speak, skipping audio playback.")
        return

    try:
        tts = gTTS(text=message, lang='en')
        local_audio_file = "temp_playback.mp3"
        tts.save(local_audio_file)

        def play_audio():
            try:
                playsound(local_audio_file)
            except Exception as e:
                print(f"Error during audio playback: {e}")
            finally:
                if os.path.exists(local_audio_file):
                    try:
                        os.remove(local_audio_file)
                    except Exception as e:
                        print(f"Warning: Could not delete temp audio file. Error: {e}")
        
        playback_thread = threading.Thread(target=play_audio)
        playback_thread.start()
        
    except Exception as e:
        print(f"An error occurred during gTTS synthesis: {e}")
        
def transcribe_audio(audio_path):
    """Converts an audio file to text using Google's free web speech API."""
    if not audio_path: return "No audio detected."
    try:
        sound = AudioSegment.from_file(audio_path)
        wav_path = "temp_audio.wav"
        sound.export(wav_path, format="wav")
        
        r = sr.Recognizer()
        with sr.AudioFile(wav_path) as source:
            audio_data = r.record(source)
            text = r.recognize_google(audio_data)
            return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return f"Speech recognition service error; {e}"
    except Exception as e:
        print(f"An error occurred during transcription: {e}")
        return "Error during transcription."
    finally:
        if os.path.exists("temp_audio.wav"):
            os.remove("temp_audio.wav")

def search_attractions(location):
    if not google_places_api_key: return {"error": "Google Places API Key not set."}
    try:
        geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={google_places_api_key}"
        geocode_response = requests.get(geocode_url)
        geocode_data = geocode_response.json()
        if geocode_data["status"] != "OK" or not geocode_data["results"]:
            return {"error": f"Location not found: {location}"}
        location_data = geocode_data["results"][0]
        lat, lng = location_data["geometry"]["location"]["lat"], location_data["geometry"]["location"]["lng"]
        params = {"location": f"{lat},{lng}", "radius": 5000, "type": "tourist_attraction", "key": google_places_api_key}
        places_response = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params)
        places_data = places_response.json()
        attractions = [{"name": p["name"], "rating": p.get("rating", "N/A"), "vicinity": p.get("vicinity", "N/A")} for p in places_data.get("results", [])[:10]]
        return {"location": location_data["formatted_address"], "attractions": attractions}
    except Exception as e:
        return {"error": str(e)}

system_message = (
    "You are a helpful and concise tourist assistant. "
    "Keep your answers to 2-3 sentences, unless the user asks for more detail. "
    "Help the user by providing good explanations about cities or places."
)

def extract_location(message):
    try:
        prompt = f"Extract the location from the following text. If no location is mentioned, return 'None'. Return only the location name.\n\nText: \"{message}\""
        response = llm.invoke(prompt)
        location = response.content.strip()
        return None if location.lower() in ['none', 'not specified'] else location
    except Exception as e:
        print(f"Error extracting location: {e}")
        return None

def chat(history):
    # Convert Gradio's history to LangChain's message format
    chat_history_messages = []
    for user_msg, bot_msg in history[:-1]: # All but the latest
        if user_msg:
            chat_history_messages.append(HumanMessage(content=user_msg))
        if bot_msg:
            chat_history_messages.append(AIMessage(content=bot_msg))

    latest_user_message = history[-1][0] if history and history[-1][0] else ""
    
    # Prepend the system message to the latest query
    full_prompt = [HumanMessage(content=system_message + "\n\nUser query: " + latest_user_message)]
    
    # Add context (location, RAG)
    location = extract_location(latest_user_message) if not (current_location and "attractions" in latest_user_message.lower()) else current_location
    if location and google_places_api_key:
        attractions_data = search_attractions(location)
        if "error" not in attractions_data and attractions_data.get("attractions"):
            attraction_context = f"\n\nHere is some context about attractions in {attractions_data['location']}:"
            for att in attractions_data["attractions"]:
                attraction_context += f"\n- {att['name']} (Rating: {att['rating']})"
            full_prompt[0].content += attraction_context # Add context to the prompt
            
    if retrieval_chain and latest_user_message:
        try:
            rag_response = retrieval_chain.invoke({"question": latest_user_message, "chat_history": chat_history_messages})
            reply = rag_response["answer"]
            unhelpful_phrases = [
                "i don't know", "i'm sorry", "i cannot answer",
                "the context does not", "based on the provided",
                "the document does not contain", "the information provided"
            ]
            is_unhelpful = any(phrase in reply.lower() for phrase in unhelpful_phrases)
            if is_unhelpful:
                print("RAG response was unhelpful, falling back to general LLM.")
                response = llm.invoke(full_prompt + chat_history_messages) 
                reply = response.content
        except Exception as e:
            print(f"RAG Error: {e}")
            response = llm.invoke(full_prompt + chat_history_messages)
            reply = response.content
    else:
        response = llm.invoke(full_prompt + chat_history_messages)
        reply = response.content
    
    talker(reply)
    return reply

# --- Load CSS ---
try:
    with open('style.css', 'r') as f:
        css = f.read()
except FileNotFoundError:
    print("Warning: style.css not found. Using default styles.")
    css = "" # Define css as an empty string to avoid errors

current_location = None

def refresh_knowledge_base():
    global vector_store, retrieval_chain
    vector_store = load_knowledge_base()
    if vector_store:
        retrieval_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=False
        )
        return "Knowledge base refreshed successfully!"
    else:
        retrieval_chain = None
        return "No PDFs found. RAG is disabled."
        
def set_location(location):
    global current_location
    if not location or not location.strip(): return "Please enter a valid location."
    current_location = location
    return f"Location set to: {location}."

with gr.Blocks(theme="default", css=css) as ui:
    with gr.Column(elem_classes="container"):
        gr.Markdown("# 🌍 Tourist Assistant", elem_classes="title")
        gr.Markdown("Your personal guide to the wonders of the world.", elem_classes="subtitle")
        gr.Image("travel.jpg", show_label=False, container=False, height=200)
        chatbot = gr.Chatbot(height=450, elem_classes="chatbot-container", type="messages")
        with gr.Row(elem_classes="input-container"):
            audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak your question", elem_id="mic-button")
            entry = gr.Textbox(placeholder="Or type your question here...", label="Text Input", show_label=False, scale=5)
        with gr.Accordion("⚙️ Settings & Tools", open=False, elem_classes="settings-accordion"):
            with gr.Tabs():
                with gr.TabItem("📍 Location"):
                    with gr.Row():
                        location_input = gr.Textbox(label="Set Current Location", placeholder="e.g., Paris, France")
                        location_btn = gr.Button("Set Location", variant="primary")
                    attractions_btn = gr.Button("Find Nearby Attractions", variant="secondary")
                    location_status = gr.Textbox(label="Location Status", interactive=False)
                with gr.TabItem("📚 Knowledge Base"):
                    refresh_btn = gr.Button("🔄 Refresh Knowledge Base", variant="primary")
                    refresh_status = gr.Textbox(label="KB Status", interactive=False)
            clear = gr.Button("🗑️ Clear Chat History")

        # --- Wire up the UI events ---
        def user(user_message, history):
            return "", history + [{"role": "user", "content": user_message}]

        def bot(history):
            history_for_backend = []
            for i in range(0, len(history), 2):
                if i + 1 < len(history):
                    user_msg = history[i]['content']
                    assistant_msg = history[i + 1]['content']
                    history_for_backend.append([user_msg, assistant_msg])
            latest_user_message = history[-1]['content']
            history_for_backend.append([latest_user_message, None])
            response = chat(history_for_backend)
            history.append({"role": "assistant", "content": response})
            return history

        def transcribe_and_chat(audio_path, history):
            if not audio_path: return history, gr.update(value=None)
            transcription = transcribe_audio(audio_path)
            history.append({"role": "user", "content": transcription})
            return bot(history), gr.update(value=None)
            
        entry.submit(user, [entry, chatbot], [entry, chatbot], queue=False).then(bot, chatbot, chatbot)
        audio_input.stop_recording(transcribe_and_chat, [audio_input, chatbot], [chatbot, audio_input])
        clear.click(lambda: [], None, chatbot, queue=False)
        refresh_btn.click(refresh_knowledge_base, None, refresh_status)
        location_btn.click(set_location, location_input, location_status).then(lambda: "", None, location_input)

        def ask_about_attractions(history):
            if not current_location:
                history.append({"role": "assistant", "content": "Please set a location first."})
                return history
            question = f"What are some attractions in {current_location}?"
            history.append({"role": "user", "content": question})
            return bot(history)
            
        attractions_btn.click(ask_about_attractions, chatbot, chatbot)

print("--- Full application code loaded. You can now launch the UI in the next cell. ---")

RAG system initialized successfully.
--- Full application code loaded. You can now launch the UI in the next cell. ---


In [6]:
# If all previous cells ran successfully, this final cell will start the Gradio web server
# and launch the user interface in your browser.

print("--- Launching Gradio UI... ---")
ui.launch(inbrowser=True, debug=True)

--- Launching Gradio UI... ---
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
